**EVALUATE Model**

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
import torch
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tabulate import tabulate
import time

# Assuming the dual-input EfficientNet model is already defined and loaded:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)  # Move the model to GPU if available

# Loss function
criterion = torch.nn.CrossEntropyLoss()

# Set the model to evaluation mode
model.eval()
correct = 0
total = 0
all_labels = []
all_predictions = []
total_loss = 0.0

eval_start_time = time.time()  # Start timing evaluation

# Adjust for test_loader returning 3 items
with torch.no_grad():
    for mfcc_inputs, mel_inputs, labels in test_loader:  # Adjusted to directly unpack all 3 elements
        mfcc_inputs, mel_inputs, labels = mfcc_inputs.to(device), mel_inputs.to(device), labels.to(device)  # Move to device

        # Forward pass with dual inputs
        outputs = model(mfcc_inputs, mel_inputs)

        # Loss computation
        loss = criterion(outputs, labels)
        total_loss += loss.item()

        _, predicted = torch.max(outputs, 1)

        # Collect all labels and predictions for metric computation
        all_labels.extend(labels.cpu().numpy())  # Move to CPU for compatibility with sklearn
        all_predictions.extend(predicted.cpu().numpy())

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

eval_end_time = time.time()  # End timing evaluation
print(f"\033[1;92mTime taken for evaluation: {eval_end_time - eval_start_time:.2f} seconds\033[0m")

# Converting to numpy arrays for sklearn
all_labels = np.array(all_labels)
all_predictions = np.array(all_predictions)

# Accuracy
accuracy = 100 * correct / total

# Precision, Recall, F1 Score
precision = precision_score(all_labels, all_predictions, average='macro')  # use 'binary' for binary classification
recall = recall_score(all_labels, all_predictions, average='macro')
f1 = f1_score(all_labels, all_predictions, average='macro')

# Confusion Matrix
conf_matrix = confusion_matrix(all_labels, all_predictions)

# Average loss
avg_loss = total_loss / len(test_loader)

# Data to be displayed in the table
data = [
    ['Accuracy', f'{accuracy:.2f}%'],
    ['Precision', f'{precision:.2f}'],
    ['Recall', f'{recall:.2f}'],
    ['F1 Score', f'{f1:.2f}'],
    ['Loss', f'{avg_loss:.4f}']
]

# Printing the table
print(tabulate(data, headers=['Metric', 'Value'], tablefmt='fancy_grid'))

# Plotting Confusion Matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=['Real', 'Fake'], yticklabels=['Real', 'Fake'])
plt.title('Confusion Matrix')
plt.ylabel('True Labels')
plt.xlabel('Predicted Labels')
plt.show()
